# Filter for Hemizygous Variants Only
- **Author(s)** - Frank Grenn
- **Date Started** - March  2021
- **Quick Description:**  filter chrY AMP-PD data to only include males, hemizygous variant, and variants with low heterozygosity

In [ ]:
import pandas as pd

In [ ]:
WRKDIR = "$PATH/chrY/y_male_only_bfiles"

In [ ]:
#get the plink files
!(module load plink; plink --bfile $PATH/bfile/chrY --make-bed --out {WRKDIR}/temp)

### update sex and pheno in fam file

In [ ]:
temp = pd.read_table(f"{WRKDIR}/chrY.fam",sep="\s+",header=None)
temp.columns = ['fid','iid','pid','mid','sex','pheno']
print(temp.shape)
print(temp.head())

In [ ]:
meta = pd.read_csv("$PATH/AMPPD_releasev2_covariates_Feb2021.csv")
print(meta.shape)
print(meta.head())

In [ ]:
tempm = pd.merge(left =temp, right = meta, left_on = 'fid', right_on = 'ID')
print(tempm.shape)

In [ ]:
tempm = tempm[['fid','iid','pid','mid','SEX','PHENO','LATEST_DX']]
print(tempm.head())

In [ ]:
tempm['PHENO'] = tempm['PHENO'].fillna(-9)
tempm['PHENO'] = tempm['PHENO'].astype(int)
set(tempm['PHENO'])

In [ ]:
print(tempm[tempm.PHENO==1].shape)
print(tempm[tempm.PHENO==2].shape)
print(tempm[tempm.PHENO==-9].shape)

In [ ]:
tempm[tempm.PHENO==-9]

In [ ]:
tempm[tempm.PHENO==1].LATEST_DX.value_counts()

In [ ]:
tempm[tempm.PHENO==2].LATEST_DX.value_counts()

In [ ]:
tempm[tempm.PHENO==-9].LATEST_DX.value_counts()

In [ ]:
tempm[['fid','iid','pid','mid','SEX','PHENO']].to_csv(f"{WRKDIR}/temp.fam",sep="\t",header=None,index=None)

### give variants unique ids
or at least set rsid variants ids to rsid_a1_a2

In [ ]:
tempbim = pd.read_table(f"{WRKDIR}/temp.bim",sep="\t",header=None)
tempbim.columns = ['chr','snp','pos','bp','a1','a2']
print(tempbim.shape)
print(tempbim.head())

In [ ]:
#see which have duplicate snp values
tempbim[tempbim.duplicated(subset = 'snp',keep=False)].sort_values('snp')

In [ ]:
len(set(tempbim['snp']))

In [ ]:
tempbim[tempbim['snp'].str.contains('rs')].head()

In [ ]:
#set snp col values with rsids to rsid_a1_a2 to make them unique
tempbim.loc[tempbim['snp'].str.contains('rs'),'snp'] = tempbim.loc[tempbim['snp'].str.contains('rs'),'snp'] + '_' + tempbim.loc[tempbim['snp'].str.contains('rs'),'a1'] + '_' + tempbim.loc[tempbim['snp'].str.contains('rs'),'a2']

In [ ]:
#check if we no longer have duplicated snp values
tempbim[tempbim.duplicated(subset = 'snp',keep=False)].sort_values('snp')

In [ ]:
tempbim.head()

In [ ]:
tempbim.tail()

In [ ]:
print(tempbim.shape)
print(len(set(tempbim['snp'].tolist())))

In [ ]:
tempbim.to_csv(f"{WRKDIR}/temp.bim",sep="\t",header=None,index=None)


### remove females

In [ ]:
!(module load plink; plink --bfile {WRKDIR}/temp --filter-males --make-bed --out {WRKDIR}/temp_males)

### check frequencies

In [ ]:
!(module load plink; plink --bfile {WRKDIR}/temp_males --freqx --out {WRKDIR}/temp_males)

In [ ]:
count = pd.read_table(f"{WRKDIR}/temp_males.frqx")
print(count.shape)
print(count.head())
print(count.tail())

### try --set-hh-missing

In [ ]:
!(module load plink; plink --bfile {WRKDIR}/temp_males --set-hh-missing --make-bed --out {WRKDIR}/temp_males_miss)

In [ ]:
!(module load plink; plink --bfile {WRKDIR}/temp_males_miss --freqx --out {WRKDIR}/temp_males_miss)

In [ ]:
miss_count = pd.read_table(f"{WRKDIR}/temp_males_miss.frqx")
print(miss_count.shape)
print(miss_count.head())
print(miss_count.tail())

### do same, but change chr to autosomal to compare counts

In [ ]:
!(module load plink; plink --bfile {WRKDIR}/temp --make-bed --out {WRKDIR}/temp22)

In [ ]:
temp22bim = pd.read_table(f"{WRKDIR}/temp22.bim",sep="\s+",header=None)
temp22bim.columns = ['chr','snp','pos','bp','a1','a2']
print(temp22bim.shape)
print(temp22bim.head())

In [ ]:
temp22bim['chr']=22

In [ ]:
temp22bim.to_csv(f"{WRKDIR}/temp22.bim",sep="\t",header=None,index=None)

In [ ]:
!(module load plink; plink --bfile {WRKDIR}/temp22 --filter-males --make-bed --out {WRKDIR}/temp22_males)

In [ ]:
!(module load plink; plink --bfile {WRKDIR}/temp22_males --freqx --out {WRKDIR}/temp22_males)

In [ ]:
recode_count = pd.read_table(f"{WRKDIR}/temp22_males.frqx")
print(recode_count.shape)
print(recode_count.head())
#print(recode_count.tail())

In [ ]:
#compare with the --set-hh-missing results
print(miss_count.shape)
print(miss_count.head())

#### so looks like the heterozygous variants were correctly set to missing

### Identify Variants with High Heterozygosity 

In [ ]:
#how many samples
!wc -l {WRKDIR}/temp22_males.fam

In [ ]:
#how many variants are heterozygous in 10% or more of the samples
recode_count[recode_count['C(HET)']/5470>=0.1].shape

In [ ]:
recode_count[recode_count['C(HET)']/5470>=0.1]

In [ ]:
bad_variants = list(set(recode_count[recode_count['C(HET)']/5470>=0.1]['SNP']))

In [ ]:
len(bad_variants)

In [ ]:
len(set(bad_variants))

In [ ]:
pd.DataFrame(data={'bad_variants':bad_variants}).to_csv(f"{WRKDIR}/high_heterozygous_variants.txt",index=None,header=None)

In [ ]:
#drop the high heterozygous variants
!(module load plink; plink --bfile {WRKDIR}/temp_males_miss --exclude {WRKDIR}/high_heterozygous_variants.txt --make-bed --out {WRKDIR}/chrY_male_hemizygous_only_het_filter)

### cleanup

In [ ]:
!(module load plink; plink --bfile {WRKDIR}/temp_males_miss --make-bed --out {WRKDIR}/chrY_male_hemizygous_only)

In [ ]:
!rm {WRKDIR}/temp*